In [1]:
# import libraries
import json
import numpy as np
import pandas as pd

In [2]:
json_file_path = 'epl-dataset/epl_2022_2023_07_02_2023.json'
with open(json_file_path, 'r') as json_file:
    data_df = pd.read_json(json_file)

# Transpose the data frame
data_df = data_df.T
# List the columns
print(list(data_df.columns))

# data frame with only the column event
event_df = data_df['event']


['event', 'matchweek', 'team1_name', 'team1_startings', 'team1_subs', 'team1_stat', 'team2_name', 'team2_startings', 'team2_subs', 'team2_stat']


In [3]:
# List the index
data_df.index

Index([75091, 75096, 75098, 75099, 75093, 75082, 75088, 75087, 75089, 75084,
       ...
       75127, 75128, 75129, 75124, 75125, 75121, 75122, 75123, 75126, 75130],
      dtype='int64', length=209)

In [4]:
# Visualize the first data
data_df.head(1)

,event,matchweek,team1_name,team1_startings,team1_subs,team1_stat,team2_name,team2_startings,team2_subs,team2_stat
75091,"[Full-time Match ends, Arsenal 0, Newcastle Un...",Matchweek 19,Arsenal,"[Aaron Ramsdale, Ben White, Gabriel Magalhães,...","[Matt Turner, Kieran Tierney, Rob Holding, Tak...","{'possession_%': '66.8', 'shots_on_target': '4...",Newcastle United,"[Nick Pope, Kieran Trippier, Sven Botman, Fabi...","[Martin Dúbravka, Jamaal Lascelles, Jamal Lewi...","{'possession_%': '33.2', 'shots_on_target': '1..."


In [5]:
# drop columns and only keep the stats columns
new_data_df = data_df
columns_to_drop = ['event', 'matchweek','team1_startings', 'team1_subs',  'team2_startings', 'team2_subs']
new_data_df.drop(columns_to_drop, axis=1, inplace=True)
new_data_df.head(1)

,team1_name,team1_stat,team2_name,team2_stat
75091,Arsenal,"{'possession_%': '66.8', 'shots_on_target': '4...",Newcastle United,"{'possession_%': '33.2', 'shots_on_target': '1..."


In [6]:
# change team1_name to home_team and team2_name to away_team
new_data_df.rename(columns={'team1_name':'home_team', 'team2_name':'away_team', 'team1_stat':'home_team_stat', 'team2_stat':'away_team_stat'}, inplace=True)
new_data_df.head(1)

,home_team,home_team_stat,away_team,away_team_stat
75091,Arsenal,"{'possession_%': '66.8', 'shots_on_target': '4...",Newcastle United,"{'possession_%': '33.2', 'shots_on_target': '1..."


In [7]:
# check for null values
new_data_df.isnull().sum()

home_team         0
home_team_stat    0
away_team         0
away_team_stat    0
dtype: int64

In [8]:
# drop the index column
new_data_df.reset_index(drop=True, inplace=True)
new_data_df.head(10)

,home_team,home_team_stat,away_team,away_team_stat
0,Arsenal,"{'possession_%': '66.8', 'shots_on_target': '4...",Newcastle United,"{'possession_%': '33.2', 'shots_on_target': '1..."
1,Everton,"{'possession_%': '48.9', 'shots_on_target': '4...",Brighton and Hove Albion,"{'possession_%': '51.1', 'shots_on_target': '8..."
2,Leicester City,"{'possession_%': '61.2', 'shots_on_target': '6...",Fulham,"{'possession_%': '38.8', 'shots_on_target': '2..."
3,Manchester United,"{'possession_%': '58.5', 'shots_on_target': '6...",Bournemouth,"{'possession_%': '41.5', 'shots_on_target': '4..."
4,Brentford,"{'possession_%': '27', 'shots_on_target': '7',...",Liverpool,"{'possession_%': '73', 'shots_on_target': '6',..."
5,Brighton and Hove Albion,"{'possession_%': '67.8', 'shots_on_target': '5...",Arsenal,"{'possession_%': '32.2', 'shots_on_target': '7..."
6,Tottenham Hotspur,"{'possession_%': '59.3', 'shots_on_target': '2...",Aston Villa,"{'possession_%': '40.7', 'shots_on_target': '4..."
7,Nottingham Forest,"{'possession_%': '27.7', 'shots_on_target': '5...",Chelsea,"{'possession_%': '72.3', 'shots_on_target': '2..."
8,West Ham United,"{'possession_%': 'West', 'shots_on_target': '6...",Brentford,"{'possession_%': 'Brentford', 'shots_on_target..."
9,Liverpool,"{'possession_%': '57', 'shots_on_target': '5',...",Leicester City,"{'possession_%': '43', 'shots_on_target': '2',..."


In [9]:
# normalize the home_team_stat column
home_team_stat_df = pd.json_normalize(new_data_df['home_team_stat'])
home_team_stat_df.rename(columns=lambda x: 'home_team'+ '_' + x, inplace=True)

# normalize the away_team_stat column
away_team_stat_df = pd.json_normalize(new_data_df['away_team_stat'])
away_team_stat_df.rename(columns=lambda x: 'away_team'+ '_' + x, inplace=True)

In [10]:
# join the home_team_stat_df and away_team_stat_df to the new_data_df
new_data_df = new_data_df.join([home_team_stat_df, away_team_stat_df])

new_data_df.head(1)

,home_team,home_team_stat,away_team,away_team_stat,home_team_possession_%,home_team_shots_on_target,home_team_shots,home_team_touches,home_team_passes,home_team_tackels,...,away_team_shots_on_target,away_team_shots,away_team_touches,away_team_passes,away_team_tackels,away_team_clearances,away_team_corners,away_team_offsides,away_team_yellow_cards,away_team_foul_conceded
0,Arsenal,"{'possession_%': '66.8', 'shots_on_target': '4...",Newcastle United,"{'possession_%': '33.2', 'shots_on_target': '1...",66.8,4,17,703,536,13,...,1,8,446,253,23,29,5,4,5,16


In [11]:
# drop the home_team_stat and away_team_stat columns
new_data_df = new_data_df.drop(['home_team_stat', 'away_team_stat'], axis=1)


In [12]:
# visualize the new_data_df
new_data_df.head(1)

,home_team,away_team,home_team_possession_%,home_team_shots_on_target,home_team_shots,home_team_touches,home_team_passes,home_team_tackels,home_team_clearances,home_team_corners,...,away_team_shots_on_target,away_team_shots,away_team_touches,away_team_passes,away_team_tackels,away_team_clearances,away_team_corners,away_team_offsides,away_team_yellow_cards,away_team_foul_conceded
0,Arsenal,Newcastle United,66.8,4,17,703,536,13,17,5,...,1,8,446,253,23,29,5,4,5,16


In [16]:
# create a new column called result
event_df = event_df.to_frame() # convert the event_df to a data frame
event_df.head(1)

,event
75091,"[Full-time Match ends, Arsenal 0, Newcastle Un..."


In [30]:
end_score_df = event_df['event'].map(lambda x: x).str[0]
end_score_df.reset_index(drop=True, inplace=True)

In [31]:
print(end_score_df)

0      Full-time Match ends, Arsenal 0, Newcastle Uni...
1      Full-time Match ends, Everton 1, Brighton and ...
2      Full-time Match ends, Leicester City 0, Fulham 1.
3      Full-time Match ends, Manchester United 3, Bou...
4        Full-time Match ends, Brentford 3, Liverpool 1.
                             ...                        
204    Full-time Match ends, Aston Villa 2, Leicester...
205    Full-time Match ends, Brentford 3, Southampton 0.
206    Full-time Match ends, Brighton and Hove Albion...
207    Full-time Match ends, Manchester United 2, Cry...
208    Full-time Match ends, Wolverhampton Wanderers ...
Name: event, Length: 209, dtype: object


In [32]:
new_data_df.loc[:, 'result'] = end_score_df

In [ ]:
new_data_df.head(10)

In [ ]:
# Create Chelsea data frame
chelsea_df = new_data_df[(new_data_df['home_team']=='Chelsea') | (new_data_df['away_team']=='Chelsea')]
chelsea_df.head(10)

In [36]:
# reset the index
chelsea_df.reset_index(drop=True, inplace=True)


In [38]:
# convert the dataframe to a csv file
chelsea_df.to_csv('epl-dataset/chelsea.csv', index=False)

